In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# --- 1. 检查 GPU (CUDA) 是否可用并打印 GPU 型号 ---
print("--- GPU (CUDA) 检查 ---")
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA 可用！正在使用 GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 数量: {torch.cuda.device_count()}")
else:
    device = torch.device("cpu")
    print("CUDA 不可用，正在使用 CPU 进行训练。")
print("-" * 30)

# --- 2. 定义一个简单的神经网络 ---
class SimpleNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# --- 3. 设置模型参数和数据 ---
input_size = 10    # 输入特征的数量
hidden_size = 20   # 隐藏层神经元的数量
output_size = 1    # 输出的数量

# 实例化模型并将其移动到指定的设备 (CPU 或 GPU)
model = SimpleNeuralNetwork(input_size, hidden_size, output_size).to(device)

# 创建一些随机的假数据和标签
# 确保数据也移动到与模型相同的设备上
num_samples = 100
X = torch.randn(num_samples, input_size).to(device)  # 输入数据
y = torch.randn(num_samples, output_size).to(device) # 对应的标签

# --- 4. 定义损失函数和优化器 ---
criterion = nn.MSELoss() # 均方误差损失，适用于回归问题
optimizer = optim.SGD(model.parameters(), lr=0.01) # 随机梯度下降优化器，学习率为 0.01

# --- 5. 训练神经网络 ---
print("\n--- 开始训练神经网络 ---")
num_epochs = 100 # 训练轮数

for epoch in range(num_epochs):
    # 将模型设置为训练模式
    model.train()

    # 前向传播
    outputs = model(X)
    loss = criterion(outputs, y)

    # 反向传播和优化
    optimizer.zero_grad() # 清除之前的梯度
    loss.backward()       # 计算梯度
    optimizer.step()      # 更新模型参数

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("--- 训练完成 ---")

# --- 6. 评估模型 (可选) ---
print("\n--- 模型评估 (在训练数据上) ---")
model.eval() # 将模型设置为评估模式
with torch.no_grad(): # 在评估时不计算梯度，节省内存和计算
    predicted_outputs = model(X)
    eval_loss = criterion(predicted_outputs, y)
    print(f"评估损失 (MSE): {eval_loss.item():.4f}")

# --- 7. 打印学到的参数 (可选) ---
# 注意：对于简单的全连接层，可以直接访问权重和偏置
print("\n--- 学到的模型参数示例 (第一层权重和偏置) ---")
# 检查fc1层的权重形状
print(f"第一层权重形状: {model.fc1.weight.shape}")
# 打印部分权重和偏置，避免输出过长
print(f"第一层权重示例:\n{model.fc1.weight.data[:2, :5]}") # 打印前2行前5列
print(f"第一层偏置示例:\n{model.fc1.bias.data[:5]}") # 打印前5个偏置


--- GPU (CUDA) 检查 ---
CUDA 可用！正在使用 GPU: NVIDIA GeForce GTX 1650 Ti
GPU 数量: 1
------------------------------

--- 开始训练神经网络 ---
Epoch [10/100], Loss: 1.0540
Epoch [20/100], Loss: 1.0379
Epoch [30/100], Loss: 1.0276
Epoch [40/100], Loss: 1.0196
Epoch [50/100], Loss: 1.0127
Epoch [60/100], Loss: 1.0064
Epoch [70/100], Loss: 1.0005
Epoch [80/100], Loss: 0.9949
Epoch [90/100], Loss: 0.9896
Epoch [100/100], Loss: 0.9845
--- 训练完成 ---

--- 模型评估 (在训练数据上) ---
评估损失 (MSE): 0.9840

--- 学到的模型参数示例 (第一层权重和偏置) ---
第一层权重形状: torch.Size([20, 10])
第一层权重示例:
tensor([[-0.0308, -0.0585,  0.2447, -0.0489, -0.1256],
        [ 0.2941, -0.3007, -0.0580,  0.0299,  0.1804]], device='cuda:0')
第一层偏置示例:
tensor([-0.0338,  0.0289,  0.1695,  0.2469, -0.1465], device='cuda:0')
